In [34]:
import geopandas as gpd
import pandas as pd
import random
import fiona
import matplotlib
import matplotlib.pyplot as plt

In [35]:
AcresTable = pd.read_csv(r"C:\Users\catan\Desktop\programming final\AcresTotalsCounty20122017.csv")
Ag2012 = []
Ag2017 = []
random.seed(0)
for row in AcresTable:
    if 'AcresHarv2012' in row:
        Ag2012.append(row)
    if 'AcresHarv2017' in row:
        Ag2017.append(row)

colnames = ['county','NumOperations2012','AcresHarv2012', 'NumOperations2017', 'AcresHarv2017']
#data = pd.read_csv('AcresTotalsCounty20122017.csv', names=colnames)

col_list2012 = ["county", "AcresHarv2012"]
colorado2012 = pd.read_csv(r"C:\Users\catan\Desktop\programming final\AcresTotalsCounty20122017.csv", usecols=col_list2012)
col_list2017 = ["county", "AcresHarv2017"]
colorado2017 = pd.read_csv(r"C:\Users\catan\Desktop\programming final\AcresTotalsCounty20122017.csv", usecols = col_list2017)

top2012 = []
top2017 = []
top5 = ['adams', 'kitcarson', 'yuma', 'weld', 'phillips']
for row in colorado2012.iterrows():
    if 'county' == top5:
        top2012.append(row)

In [75]:
datatable = {'county': [], 'AcresHarv2012': [], 'AcresHarv2017': []}
#df = pd.DataFrame(columns = ['county', 'AcresHarv2012', 'AcresHarv2017'])

from csv import DictReader
Names = ["yuma", "weld", "phillips", "kit carson", "adams"]

with open(r"C:\Users\catan\Documents\GitHub\final-programming-project\AcresTotalsCounty20122017.csv", newline='') as csvfile:
    reader = DictReader(csvfile)
    #rows = [row for row in reader if row['county'] in Names]
    
    for row in reader:
        if row['county'] in Names:
            county = row['county']
            AcresHarv2012 = int(row['AcresHarv2012'])
            AcresHarv2017 = int(row['AcresHarv2017'])
            
            datatable['county'].append(county)
            datatable['AcresHarv2012'].append(AcresHarv2012)
            datatable['AcresHarv2017'].append(AcresHarv2017)
    
HarvGdf = gpd.GeoDataFrame(datatable)

HarvGdf

ValueError: invalid literal for int() with base 10: '283,538'

In [97]:
def join_func(join, left, right, variable):
    "This function performs a join between two tables or variables."

    join = left.merge(right, on=variable)

In [98]:
# precipitation join

cty2012prcp = pd.read_csv(r"C:\Users\catan\Documents\GitHub\final-programming-project\cty2012prcp.csv")
cty2017prcp = pd.read_csv(r"C:\Users\catan\Documents\GitHub\final-programming-project\cty2017prcp.csv")

#prcp_join = cty2012prcp.merge(cty2017prcp, on='county')
join_func(prcp_join, cty2012prcp, cty2017prcp, 'county')

In [99]:
# joining precipitation and harvest data

#all_data = prcp_join.merge(HarvGdf, on='county')
join_func(all_data, prcp_join, HarvGdf, 'county')
all_data

,county,avg_prcp2012,avg_prcp2017,AcresHarv2012,AcresHarv2017,prcp_change,harv_change


In [55]:
# creating percent change columns

all_data['prcp_change'] = ((all_data['avg_prcp2017'] - all_data['avg_prcp2012']) / all_data['avg_prcp2012']) * 100
all_data['harv_change'] = ((all_data['AcresHarv2017'] - all_data['AcresHarv2012']) / all_data['AcresHarv2012']) * 100

all_data

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [87]:
def perc_change (data2017, data2012, change, csv_output):
    # this function calculates percent change between two variables

    change = ((data2017 - data2012) / data2012) * 100
    df = pd.DataFrame(change)
    df.to_csv(csv_output)

    return df

In [90]:
#all_data['prcp_change'] = ((all_data['avg_prcp2017'] - all_data['avg_prcp2012']) / all_data['avg_prcp2012']) * 100

all_data['prcp_change'] = []
all_data['harv_change'] = []

perc_change(all_data['avg_prcp2017'], all_data['avg_prcp2012'], all_data['prcp_change'], 'final_project.csv')

,0


In [89]:
all_data

,county,avg_prcp2012,avg_prcp2017,AcresHarv2012,AcresHarv2017,prcp_change,harv_change


In [40]:
# exporting final table to csv

all_data.to_csv(r"C:\Users\catan\Documents\GitHub\final-programming-project\final_proj.csv", index = False)

In [41]:
# concluding print statement

print("The county with the largest precipitation change is Yuma. The county with the largest acres harvest change is Kit Carson.")

The county with the largest precipitation change is Yuma. The county with the largest acres harvest change is Kit Carson.


In [91]:
# map creation

fp = r"C:\Users\catan\Documents\GitHub\final-programming-project\Colorado_County_Boundaries.shp"
#SHAPE_RESTORE_SHX=YES fiona.info Colorado_County_Boundaries1.shp
map_df = gpd.read_file(fp)
map_df['COUNTY'] = map_df['COUNTY'].str.lower()
df = pd.read_csv(r"C:\Users\catan\Documents\GitHub\final-programming-project\final_proj.csv", header=0)
merged = map_df.set_index('COUNTY').join(df.set_index('county'))
merged = map_df.merge(df, left_on='COUNTY', right_on='county')
variable = merged['prcp_change']
vmin, vmax = 0, 100
fig, ax = plt.subplots(1, figsize = (10,6))
merged.plot(column = variable, cmap = 'twilight', linewidth = 0.8, ax = ax, edgecolor = '0.8')
print(merged)
ax.axis('off')
ax.set_title('Colorado Counties Precipitation Change from 2012 to 2017', fontdict={'fontsize': '25', 'fontweight' : '3'})
ax.annotate('Data source: Colorado State University, Warner College; USDA', xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
ax.annotate('Created by Hannah Shook, Mallory Prentiss, and Rachel Tarbet', xy=(0.1, .12),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

sm = plt.cm.ScalarMappable(cmap='twilight', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
plt.colorbar(sm,fraction=0.046, pad=0.0000001)

fig.savefig(r'C:\Users\catan\Desktop\programming final\map_export.png', dpi=300)



KeyError: "None of ['county'] are in the columns"